#### Exercise 6 (4 points): Markov Switching Autoregression Models

__Do It Yourself__. 

1. Go to Bloomberg or Yahoo Finance to download the daily commodity price of your choice from **__January 1, 2020__** until **__September 30, 2023__**. 

2. Compute its continous daily, monthly and quarterly returns. Specify a markov switching autoregression model on these 3 time series from the beginning of your sample till **__December 31, 2022__**. 

3. Next, do some forecasts over the next $3$ quarters,
$9$ months and the subsequent days (for this daily prediction, please predict $20$-step ahead forecasts and next re-estimaste your model by including the true realizations and predict again $20$-step ahead
forecasts, do it till the end of the sample). You may use *statmodels* to do it.

**Be careful, you have to figure out which SARIMA-GARCH model you should apply first on the time series before adding a Markov Switching process.**

In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import yfinance as yf
from datetime import datetime
import itertools

sns.set_theme(style="darkgrid")

In [17]:
# Ticker list
ticker = "ZW=F"

# Date range
start = datetime(2020, 1, 1)
end = datetime(2023, 9, 30)

# Get data
data = pd.DataFrame(
    yf.download(ticker, start=start, end=end, interval="1d")["Adj Close"],
    columns=["Adj Close"],
)

# Rename the column to the asset name wheat
data.rename(columns={"Adj Close": "Wheat"}, inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [22]:
# Function to compute daily / monthly / quarterly continuous returns

def compute_returns(df, freq, asset):
    if freq == "daily":
        df[freq + " returns " + asset] = df[asset].pct_change()
    elif freq == "monthly":
        df[freq + " returns " + asset] = df[asset].pct_change(21)
    elif freq == "quarterly":
        df[freq + " returns " + asset] = df[asset].pct_change(63)
    else:
        print("Invalid frequency")
    return df


# Compute daily / monthly / quarterly continuous returns
for cols in data.columns:
    for freq in ["daily", "monthly", "quarterly"]:
        data = compute_returns(data, freq, cols)

In [23]:
data = data.dropna()
data

,wheat,daily returns wheat,monthly returns wheat,quarterly returns wheat
Date,,,,
2020-04-02,541.75,-0.015448,0.038830,-0.033021
2020-04-03,549.25,0.013844,0.047687,-0.009468
2020-04-06,555.75,0.011834,0.066187,0.010455
2020-04-07,549.25,-0.011696,0.051699,-0.001817
2020-04-08,548.25,-0.001821,0.040816,-0.008141
...,...,...,...,...
2023-09-25,589.00,0.016393,-0.024834,-0.186745
2023-09-26,589.00,0.000000,-0.007164,-0.140146
2023-09-27,579.50,-0.016129,-0.014456,-0.116279
